# Aula: Conversão de Tipos, Datas e Timezones em Python

Este notebook foi preparado para praticar:

- Conversão entre tipos numéricos e strings
- Conversão e validação de datas
- Trabalho com timestamps
- Trabalho com fusos horários (timezones) usando `pytz`

Use as células de **Exercício** para os alunos resolverem e as células de **Gabarito** como solução.

In [67]:
import pandas as pd
import datetime
import re
import pytz

## 1. Dados base

Nesta seção vamos criar um `DataFrame` de exemplo que será usado em todos os exercícios.

In [68]:
data = {
    "id": [1, 2, 3, 4, 5],
    "age_str": ["25", "30", "not_available", "18", "40"],
    "salary_float": [2500.5, 3200.0, 4100.75, 1800.0, 5000.0],
    "salary_str": ["2500.50", "3200.00", "4100.75", "1800.00", "5000.00"],
    "score_int": [10, 8, 9, 7, 10],
    "score_str": ["10", "8", "9", "7", "10"],
    "order_date_str": ["2022-01-01", "2022-02-15", "2022-13-01", "2020-02-29", "2021-12-31"],
    "event_timestamp": [1609459200.0, 1640995200.0, 1672531200.0, 1582934400.0, 1640908800.0],
    "leap_date_str": ["2024-02-29", "2022-02-29", "2020-02-29", "2019-02-29", "2000-02-29"],
    "timezone_name": [
        "UTC",
        "America/Los_Angeles",
        "Europe/Berlin",
        "America/Sao_Paulo",
        "Asia/Tokyo",
    ],
}

df = pd.DataFrame(data)
df

,id,age_str,salary_float,salary_str,score_int,score_str,order_date_str,event_timestamp,leap_date_str,timezone_name
0,1,25,2500.50,2500.50,10,10,2022-01-01,1.609459e+09,2024-02-29,UTC
1,2,30,3200.00,3200.00,8,8,2022-02-15,1.640995e+09,2022-02-29,America/Los_Angeles
2,3,not_available,4100.75,4100.75,9,9,2022-13-01,1.672531e+09,2020-02-29,Europe/Berlin
3,4,18,1800.00,1800.00,7,7,2020-02-29,1.582934e+09,2019-02-29,America/Sao_Paulo
4,5,40,5000.00,5000.00,10,10,2021-12-31,1.640909e+09,2000-02-29,Asia/Tokyo


## 2. Conversões Numéricas ↔ String

Nesta seção vamos praticar conversões de tipos numéricos para string e vice-versa.

### Exercício 1 – Converter numérico para string

**Enunciado**

A partir do `df`, cria uma nova coluna `salary_text` que seja a versão em texto (`str`) da coluna `salary_float`.

_Tente resolver na célula abaixo antes de olhar o gabarito._

In [69]:
# Sua solução aqui
df["salary_text"] = df["salary_float"].astype(str)
df[["salary_float", "salary_text"]]

,salary_float,salary_text
0,2500.50,2500.5
1,3200.00,3200.0
2,4100.75,4100.75
3,1800.00,1800.0
4,5000.00,5000.0


### Exercício 2 – Converter string para `int`

**Enunciado**

Converte a coluna `score_str` para inteiro e guarda o resultado numa nova coluna chamada `score_int_from_str`.

In [70]:
# Sua solução aqui
df["score_int_from_str"] = df["score_str"].astype(int)
df[["score_str", "score_int_from_str"]]

,score_str,score_int_from_str
0,10,10
1,8,8
2,9,9
3,7,7
4,10,10


### Exercício 3 – Converter `int` para `float`

**Enunciado**

Converte a coluna `score_int` para `float` e guarda numa nova coluna `score_float`.

In [71]:
# Sua solução aqui
df["score_float"] = df["score_int"].astype(float)
df[["score_int", "score_float"]]

,score_int,score_float
0,10,10.0
1,8,8.0
2,9,9.0
3,7,7.0
4,10,10.0


### Exercício 4 – Converter string para `float`

**Enunciado**

Converte a coluna `salary_str` para `float` e guarda numa nova coluna `salary_from_str_float`.

In [72]:
# Sua solução aqui
df["salary_from_str_float"] = df["salary_str"].astype(float)
df[["salary_str", "salary_from_str_float"]]

,salary_str,salary_from_str_float
0,2500.50,2500.50
1,3200.00,3200.00
2,4100.75,4100.75
3,1800.00,1800.00
4,5000.00,5000.00


### Exercício 5 – Tratar erro ao converter string para número

**Enunciado**

A coluna `age_str` tem valores válidos (`"25"`, `"30"`, `"18"`, `"40"`) e um valor inválido (`"not_available"`).

Converte `age_str` para numérico, mas quando não for possível, coloca `NaN` em vez de erro. Guarda o resultado na coluna `age_int`.

_Dica: usa `pd.to_numeric` com `errors="coerce"`._

In [73]:
# Sua solução aqui
df["age_int"] = pd.to_numeric(df["age_str"], errors="coerce")
df[["age_str", "age_int"]]

,age_str,age_int
0,25,25.0
1,30,30.0
2,not_available,NaN
3,18,18.0
4,40,40.0


## 3. Datas e Timestamps

Agora vamos trabalhar com strings de datas, validação de formatos e timestamps (segundos desde 1970).

### Exercício 6 – Converter coluna de string para `datetime`

**Enunciado**

A coluna `order_date_str` contém datas em formato `"YYYY-MM-DD"`, mas uma delas é inválida (`"2022-13-01"`).

1. Converte `order_date_str` para `datetime` usando `pd.to_datetime` com o formato `"%Y-%m-%d"`.
2. Se a data for inválida, o valor deve virar `NaT` (não lançar erro).
3. Guarda o resultado numa nova coluna `order_date`.

In [74]:
# Sua solução aqui
df["order_date"] = pd.to_datetime(df["order_date_str"], errors="coerce")
df[["order_date_str", "order_date"]]

,order_date_str,order_date
0,2022-01-01,2022-01-01
1,2022-02-15,2022-02-15
2,2022-13-01,NaT
3,2020-02-29,2020-02-29
4,2021-12-31,2021-12-31


### Exercício 7 – Validar formato da string de data com regex antes de converter

**Enunciado**

Agora, em vez de usar apenas `pd.to_datetime`, queremos simular a lógica com regex.

1. Cria uma função `parse_date_safe(date_str)` que:
   - verifica se `date_str` respeita o padrão `"YYYY-MM-DD"` usando `re.match(r"^\d{4}-\d{2}-\d{2}$", date_str)`;
   - se corresponder ao padrão, tenta converter usando `datetime.datetime.strptime(date_str, "%Y-%m-%d").date()`;
   - se o formato não bater **ou** a data for inválida, retorna a string `"invalid date format"`.
2. Aplica essa função na coluna `order_date_str` e guarda o resultado em `order_date_parsed_manual`.

In [75]:
# Sua solução aqui
def parse_date_safe(date_str):
    if re.match(r"^\d{4}-\d{2}-\d{2}$", date_str):
        try:
            return datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
        except:
            return "invalid date format"
    else:
        return "invalid date format"
    
df["order_date_parsed_manual"] = df["order_date_str"].apply(parse_date_safe)
df[["order_date_str", "order_date_parsed_manual"]]

,order_date_str,order_date_parsed_manual
0,2022-01-01,2022-01-01
1,2022-02-15,2022-02-15
2,2022-13-01,invalid date format
3,2020-02-29,2020-02-29
4,2021-12-31,2021-12-31


### Exercício 8 – Converter timestamp para `datetime` UTC

**Enunciado**

A coluna `event_timestamp` contém timestamps em segundos desde 1970.

1. Converte `event_timestamp` para `datetime` em UTC usando `pd.to_datetime` com `unit="s"` e `utc=True`.
2. Guarda o resultado na nova coluna `event_datetime_utc`.

In [76]:
# Sua solução aqui
df["event_datetime_utc"] = pd.to_datetime(df["event_timestamp"], unit="s", utc=True)
df[["event_timestamp", "event_datetime_utc"]]

,event_timestamp,event_datetime_utc
0,1.609459e+09,2021-01-01 00:00:00+00:00
1,1.640995e+09,2022-01-01 00:00:00+00:00
2,1.672531e+09,2023-01-01 00:00:00+00:00
3,1.582934e+09,2020-02-29 00:00:00+00:00
4,1.640909e+09,2021-12-31 00:00:00+00:00


### Exercício 9 – Validar datas de ano bissexto

**Enunciado**

A coluna `leap_date_str` tem datas como `"2024-02-29"`, `"2022-02-29"`, etc.

1. Cria uma função `validate_leap_date(date_str)` que:
   - tenta converter `date_str` para `datetime.date` usando `datetime.datetime.strptime(date_str, "%Y-%m-%d").date()`; se der erro, retorna `"invalid date"`;
   - se converter, verifica se o ano é bissexto com a regra: ano divisível por 4 e (não divisível por 100 ou divisível por 400);
   - se a data for 29 de fevereiro num ano **não** bissexto, retorna `"invalid date"`;
   - caso contrário, retorna um `datetime.datetime` às 00:00 (`datetime.datetime.combine(date, datetime.time.min)`).
2. Aplica essa função em `leap_date_str` e guarda o resultado em `leap_parsed_or_invalid`.

In [77]:
# Sua solução aqui
def validate_leap_date(date_str):
    try:
        date = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
    except:
        return "invalid date"
    
    year = date.year
    month = date.month
    day = date.day

    bissexto = (year % 4 == 0) and (year % 100 != 0 or year % 400 == 0)

    if day == 29 and month == 2 and not bissexto:
        return "invalid date"
    else:
        return datetime.datetime.combine(date, datetime.time.min)

df["leap_parsed_or_invalid"] = df["leap_date_str"].apply(validate_leap_date)
df[["leap_date_str", "leap_parsed_or_invalid"]]

,leap_date_str,leap_parsed_or_invalid
0,2024-02-29,2024-02-29 00:00:00
1,2022-02-29,invalid date
2,2020-02-29,2020-02-29 00:00:00
3,2019-02-29,invalid date
4,2000-02-29,2000-02-29 00:00:00


### Exercício 10 – Concatenar string com data do DataFrame

**Enunciado**

Queremos criar uma frase com a data do pedido, por exemplo: `"Order 1 date is 2022-01-01"`.

1. Usa a coluna `id` e a coluna `order_date`.
2. Cria uma nova coluna `order_message` com o texto: `"Order <id> date is <YYYY-MM-DD>"`.
3. Usa `strftime` ou `astype(str)` para evitar o `TypeError`.

In [78]:
# Sua solução aqui
df["order_message"] = ("Order " + df["id"].astype(str) + " date is " + df["order_date"].dt.strftime("%Y-%m-%d"))
df[["id", "order_date" ,"order_message"]]

,id,order_date,order_message
0,1,2022-01-01,Order 1 date is 2022-01-01
1,2,2022-02-15,Order 2 date is 2022-02-15
2,3,NaT,NaN
3,4,2020-02-29,Order 4 date is 2020-02-29
4,5,2021-12-31,Order 5 date is 2021-12-31


## 4. Timezones (Fusos Horários)

Agora vamos trabalhar com fusos horários usando `pytz` e as colunas `event_datetime_utc` e `timezone_name`.

### Exercício 11 – Adicionar fuso horário a uma data sem timezone

**Enunciado**

Queremos simular o caso:

```python
date = datetime.datetime(2022, 1, 1, 0, 0, 0)
utc_date = pytz.utc.localize(date)
```

Usando o DataFrame:

1. Cria uma nova coluna `order_datetime_naive` que seja um `datetime` sem fuso horário, combinando a coluna `order_date` com horário `00:00:00`.
2. Cria uma nova coluna `order_datetime_utc` que seja a mesma data, mas com fuso UTC, usando `pytz.utc.localize`.

_Aplique apenas onde `order_date` não for `NaT`._

In [79]:
# Sua solução aqui
def make_naive_midnight(date_val):
    if pd.isna(date_val):
        return pd.NaT
    return datetime.datetime.combine(date_val, datetime.time.min)
df["order_datetime_naive"] = df["order_date"].apply(make_naive_midnight)

def localize_utc(dt_val):
    if pd.isna(dt_val):
        return pd.NaT
    return pytz.utc.localize(dt_val)
df["order_datetime_utc"] = df["order_datetime_naive"].apply(localize_utc)

df[["order_datetime_naive", "order_datetime_utc"]]

,order_datetime_naive,order_datetime_utc
0,2022-01-01,2022-01-01 00:00:00+00:00
1,2022-02-15,2022-02-15 00:00:00+00:00
2,NaT,NaT
3,2020-02-29,2020-02-29 00:00:00+00:00
4,2021-12-31,2021-12-31 00:00:00+00:00


### Exercício 12 – Converter de UTC para o fuso horário da linha

**Enunciado**

Usando a coluna `event_datetime_utc` (criada no Exercício 8) e `timezone_name`:

1. Cria uma função `convert_to_timezone(row)` que:
   - pega `row["event_datetime_utc"]` e `row["timezone_name"]`;
   - converte o datetime de UTC para o fuso horário indicado usando `astimezone(pytz.timezone(...))`.
2. Aplica essa função linha a linha (`axis=1`) e guarda o resultado na coluna `event_local_datetime`.

In [80]:
# Sua solução aqui
def convert_to_timezone(row):
    dt_utc = row["event_datetime_utc"]
    tz = row["timezone_name"]

    try:
        tz_obj = pytz.timezone(tz)
        return dt_utc.astimezone(tz_obj)
    except:
        return pd.NaT             

df["event_local_datetime"] = df.apply(convert_to_timezone, axis=1)
df[["event_datetime_utc", "event_local_datetime"]]

,event_datetime_utc,event_local_datetime
0,2021-01-01 00:00:00+00:00,2021-01-01 00:00:00+00:00
1,2022-01-01 00:00:00+00:00,2021-12-31 16:00:00-08:00
2,2023-01-01 00:00:00+00:00,2023-01-01 01:00:00+01:00
3,2020-02-29 00:00:00+00:00,2020-02-28 21:00:00-03:00
4,2021-12-31 00:00:00+00:00,2021-12-31 09:00:00+09:00


### Exercício 13 – Formatar a data com fuso horário como string

**Enunciado**

Usando a coluna `event_local_datetime`:

1. Cria uma coluna `event_local_str` formatando a data no formato `"YYYY-MM-DD HH:MM:SS Z"`.
2. Usa `strftime("%Y-%m-%d %H:%M:%S %Z")`.

In [81]:
# Sua solução aqui
def format_with_tz(dt_val):
    if pd.isna(dt_val):
        return None
    return dt_val.strftime("%Y-%m-%d %H:%M:%S %Z")

df["event_local_str"] = df["event_local_datetime"].apply(format_with_tz)
df[["event_local_str","event_local_datetime"]]

,event_local_str,event_local_datetime
0,2021-01-01 00:00:00 UTC,2021-01-01 00:00:00+00:00
1,2021-12-31 16:00:00 PST,2021-12-31 16:00:00-08:00
2,2023-01-01 01:00:00 CET,2023-01-01 01:00:00+01:00
3,2020-02-28 21:00:00 -03,2020-02-28 21:00:00-03:00
4,2021-12-31 09:00:00 JST,2021-12-31 09:00:00+09:00


### Exercício 14 – Verificar se o evento está dentro do horário de trabalho local

**Enunciado**

Considera que o horário de trabalho local é de **09:00** a **17:00** no fuso horário da própria linha (já refletido em `event_local_datetime`).

1. Para cada linha, cria dois novos `datetime`:
   - `start_time` = mesma data local, 09:00;
   - `end_time`   = mesma data local, 17:00;
2. Verifica se `event_local_datetime` está entre `start_time` e `end_time` (inclusive).
3. Cria uma coluna booleana `within_work_hours` com `True` ou `False`.

In [83]:
# Sua solução aqui
def is_within_work_hours(dt_local):
    if pd.isna(dt_local):
        return False
    
    ts = pd.Timestamp(dt_local)
    midnight = ts.normalize()
    start = midnight + pd.Timedelta(hours=9)
    end = midnight + pd.Timedelta(hours=17)

    return (ts >= start) and (ts <= end)

df["within_work_hours"] = df["event_local_datetime"].apply(is_within_work_hours)

df[["event_local_datetime", "within_work_hours"]]

,event_local_datetime,within_work_hours
0,2021-01-01 00:00:00+00:00,False
1,2021-12-31 16:00:00-08:00,True
2,2023-01-01 01:00:00+01:00,False
3,2020-02-28 21:00:00-03:00,False
4,2021-12-31 09:00:00+09:00,True
